In [3]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chatterbot 1.0.4 requires python-dateutil<2.8,>=2.7, but you have python-dateutil 2.9.0.post0 which is incompatible.
chatterbot 1.0.4 requires sqlalchemy<1.3,>=1.2, but you have sqlalchemy 2.0.35 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Library

In [1]:
from langchain_community.document_loaders import WikipediaLoader

## Load Model

In [2]:
docs = WikipediaLoader(query="East_kalimantan",  load_max_docs=2, lang='id').load() 

In [3]:
docs[0].metadata # metadata of the first document

{'title': 'Kalimantan',
 'summary': 'Kalimantan (toponim: Kalamantan, Calémantan, Kalémantan, Kelamantan, Kilamantan, Klamantan, Klémantan, K\'lemantan, Quallamontan), atau juga disebut Borneo oleh dunia internasional, adalah pulau terbesar ketiga di dunia yang terletak di sebelah utara Pulau Jawa dan di sebelah barat Pulau Sulawesi. Pulau Kalimantan dibagi menjadi wilayah Indonesia (73%), Malaysia (26%), dan Brunei (1%). Pulau Kalimantan terkenal dengan julukan "Pulau Seribu Sungai" karena banyaknya sungai yang mengalir di pulau ini.\nBorneo—yang berasal dari nama kesultanan Brunei—adalah nama yang dipakai oleh kolonial Inggris dan Belanda untuk menyebut pulau ini secara keseluruhan, sedangkan Kalimantan adalah nama yang digunakan oleh penduduk bagian timur pulau yang sekarang termasuk wilayah Indonesia.\nWilayah utara pulau ini (Sabah, Brunei, Sarawak) untuk Malaysia dan Brunei Darussalam. Sementara untuk Indonesia wilayah Utara, adalah provinsi Kalimantan Utara.\nDalam arti luas "Ka

## Build RAG

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # import the text splitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap= 200) # chunk_size is the size of each chunk, chunk_overlap is the overlap between chunks
all_split= text_splitter.split_documents(docs) # split the text into chunks
all_split[0].page_content # metadata of the first chunk

'Kalimantan (toponim: Kalamantan, Calémantan, Kalémantan, Kelamantan, Kilamantan, Klamantan, Klémantan, K\'lemantan, Quallamontan), atau juga disebut Borneo oleh dunia internasional, adalah pulau terbesar ketiga di dunia yang terletak di sebelah utara Pulau Jawa dan di sebelah barat Pulau Sulawesi. Pulau Kalimantan dibagi menjadi wilayah Indonesia (73%), Malaysia (26%), dan Brunei (1%). Pulau Kalimantan terkenal dengan julukan "Pulau Seribu Sungai" karena banyaknya sungai yang mengalir di pulau ini.\nBorneo—yang berasal dari nama kesultanan Brunei—adalah nama yang dipakai oleh kolonial Inggris dan Belanda untuk menyebut pulau ini secara keseluruhan, sedangkan Kalimantan adalah nama yang digunakan oleh penduduk bagian timur pulau yang sekarang termasuk wilayah Indonesia.\nWilayah utara pulau ini (Sabah, Brunei, Sarawak) untuk Malaysia dan Brunei Darussalam. Sementara untuk Indonesia wilayah Utara, adalah provinsi Kalimantan Utara.'

In [5]:
len(all_split) # number of chunks (ada 12 potongan)

12

## Embedding

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="")


c:\Users\ShaneZolDyCk\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Store Text

In [7]:
from langchain_community.vectorstores import FAISS

# Create the FAISS vectorstore from documents
vectorstore = FAISS.from_documents(documents=all_split, embedding=embeddings)

# Convert vectorstore to a retriever with the correct search_type argument
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})  # "k" is the number of similar documents to retrieve


## LLM

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
 # create the generative AI model
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key="")

## Prompt Template

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
    Answer the question as detailed as possible from the provided context. make sure to provide all the details\n                                       
    Context: {context}\n
    Pertaanyaan: {question}

 """)


## Inference

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Jelaskan Geografi Kalimantan Timur"):
    print(chunk, end="", flush=True)

**Geografis Kalimantan Timur**

* **Luas:** 127.346,92 km²
* **Populasi:**
    * 2020: 3.941.766 jiwa
    * Pertengahan 2024 (proyeksi): 4.050.079 jiwa
* **Kepadatan Penduduk:** Terendah keempat di Indonesia
* **Ibu Kota:** Kota Samarinda
* **Letak:** Pulau Kalimantan, Indonesia

**Batas Wilayah:**

* Utara: Kalimantan Utara, Ibu Kota Nusantara
* Timur: Sulawesi
* Selatan: Kalimantan Selatan, Kalimantan Barat
* Barat: Kalimantan Tengah
* Timur Laut: Sarawak (Malaysia Timur)

**Ciri Fisik:**

* Didominasi hutan hujan tropis
* Memiliki sungai yang banyak, sehingga dijuluki "Pulau Seribu Sungai"
* Memiliki beragam kerajaan di masa lalu, seperti Kerajaan Kutai, Kesultanan Kutai Kartanegara ing Martadipura, Kesultanan Pasir, dan Kesultanan Berau